In [1]:
wd <- dirname(getwd())
source(paste0(wd,"/mission_control/treasure_map.R"))
source(paste0(wd,"/mission_control/isofox_help.R"))

library(dplyr)
library(tidyverse)

Warning message:
“package ‘dplyr’ was built under R version 3.6.3”
Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ────────────────────────────────────────────────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.3.3     ✔ readr   1.3.1
✔ tibble  3.1.2     ✔ purrr   0.3.4
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ ggplot2 3.3.3     ✔ forcats 0.5.1
Warning message:
“package ‘ggplot2’ was built under R version 3.6.3”Warning message:
“package ‘tibble’ was built under R version 3.6.3”Warning message:
“package ‘tidyr’ was built under R version 3.6.3”Warning message:
“package ‘purrr’ was built under R version 3.6.3”Warning message:
“package ‘forcats’ was built under R version 3.6.3”── Conflicts ─────────────────────────

In [2]:
z <- read.csv(paste0( TMP_DIR,"clinical_ready.csv"), header = TRUE, sep = ",", stringsAsFactors = FALSE)
a <- read.csv(paste0( TMP_DIR,"isofox_RawTPM_features.csv"), header = TRUE, sep = ",", stringsAsFactors = FALSE)
b <- read.csv(paste0( TMP_DIR,"isofox_RawTPM_full_features.csv"), header = TRUE, sep = ",", stringsAsFactors = FALSE)

In [ ]:
c <- read.csv(paste0( TMP_DIR,"isofox_AdjTPM_features.csv"), header = TRUE, sep = ",", stringsAsFactors = FALSE)
d <- read.csv(paste0( TMP_DIR,"isofox_AdjTPM_full_features.csv"), header = TRUE, sep = ",", stringsAsFactors = FALSE)

In [4]:
dim(a); dim(b); dim(c); dim(d)

[1]  2248 18733

In [20]:
a_sum <- data.frame( sample = a %>% pull(sampleId), sums_a = apply( a %>% select(-sampleId), 1, sum))
b_sum <- data.frame( sample = b %>% pull(sampleId), sums_b = apply( b %>% select(-sampleId), 1, sum))
c_sum <- data.frame( sample = c %>% pull(sampleId), sums_c = apply( c %>% select(-sampleId), 1, sum))
d_sum <- data.frame( sample = d %>% pull(sampleId), sums_d = apply( d %>% select(-sampleId), 1, sum))

In [52]:
idx <- which(colnames(b) %in% colnames(a))
present <- b[,idx[-1]]
missing <- b[,-idx]

present_sums <- sort(apply(present, 2, mean), decreasing = TRUE)
missing_sums <- sort(apply(missing, 2, mean), decreasing = TRUE)

present_checks <- data.frame( amt = present_sums, pct = present_sums/1000000, tot_exp = cumsum(present_sums/1000000))
missing_checks <- data.frame( amt = missing_sums, pct = missing_sums/1000000, tot_exp = cumsum(missing_sums/1000000))

In [61]:
missing_checks[1:5,]; present_checks[1:5,]

,amt,pct,tot_exp
isofox_RN7SL2,274949.98,0.27494998,0.2749500
isofox_RN7SL1,204344.52,0.20434452,0.4792945
isofox_RN7SK,112827.37,0.11282737,0.5921219
isofox_RN7SL4P,57316.13,0.05731613,0.6494380
isofox_RN7SL5P,48592.98,0.04859298,0.6980310


,amt,pct,tot_exp
isofox_FTH1,747.1163,0.0007471163,0.0007471163
isofox_EEF1A1,713.3980,0.0007133980,0.0014605143
isofox_ACTB,698.6368,0.0006986368,0.0021591511
isofox_RPL37A,686.3139,0.0006863139,0.0028454650
isofox_B2M,670.3777,0.0006703777,0.0035158427


In [62]:
idx <- which(colnames(d) %in% colnames(c))
present <- d[,idx[-1]]
missing <- d[,-idx]

present_sums <- sort(apply(present, 2, mean), decreasing = TRUE)
missing_sums <- sort(apply(missing, 2, mean), decreasing = TRUE)

present_checks <- data.frame( amt = present_sums, pct = present_sums/1000000, tot_exp = cumsum(present_sums/1000000))
missing_checks <- data.frame( amt = missing_sums, pct = missing_sums/1000000, tot_exp = cumsum(missing_sums/1000000))

In [63]:
missing_checks[1:5,]; present_checks[1:5,]

,amt,pct,tot_exp
isofox_RN7SL2,1741814.9,1.7418149,1.741815
isofox_RN7SL1,1296960.7,1.2969607,3.038776
isofox_RN7SK,510066.2,0.5100662,3.548842
isofox_RN7SL4P,364069.1,0.3640691,3.912911
isofox_RN7SL5P,320208.2,0.3202082,4.233119


,amt,pct,tot_exp
isofox_FTH1,3354.188,0.003354188,0.003354188
isofox_ACTB,3071.009,0.003071009,0.006425197
isofox_EEF1A1,2983.380,0.002983380,0.009408578
isofox_FTL,2932.656,0.002932656,0.012341234
isofox_B2M,2832.041,0.002832041,0.015173275


In [57]:
present_checks[which(rownames(present_checks) == "isofox_CXCL9"),]

,amt,pct,tot_exp
isofox_CXCL9,5.595663,5.595663e-06,0.1312679


In [21]:
combined <- (
    a_sum 
        %>% inner_join( b_sum, by = "sample") 
        %>% inner_join( c_sum, by = "sample")
        %>% inner_join( d_sum, by = "sample")
)

In [23]:
summary(combined)

              sample         sums_a           sums_b            sums_c      
 CPCT02010255TII :   1   Min.   :  2477   Min.   : 999865   Min.   :383407  
 CPCT02010256TII :   1   1st Qu.:112354   1st Qu.: 999967   1st Qu.:635468  
 CPCT02010267TII :   1   Median :151793   Median :1000001   Median :681325  
 CPCT02010267TIII:   1   Mean   :152565   Mean   :1000001   Mean   :676155  
 CPCT02010291TII :   1   3rd Qu.:191955   3rd Qu.:1000033   3rd Qu.:720494  
 CPCT02010292TII :   1   Max.   :458894   Max.   :1000163   Max.   :970622  
 (Other)         :2242                                                      
     sums_d         
 Min.   :  1623178  
 1st Qu.:  3620647  
 Median :  4514778  
 Mean   :  5695457  
 3rd Qu.:  5908144  
 Max.   :154804555  
                    